In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy.coordinates as coord
import astropy.units as u
import sys
from astropy.table import Table

if './SelfCalGroupFinder/py/' not in sys.path:
    sys.path.append('./SelfCalGroupFinder/py/')
from pyutils import *
import plotting as pp
from dataloc import *
import catalog_definitions as cat
import groupcatalog as gc
from nnanalysis import *

%load_ext autoreload
%autoreload 2

In [4]:
blue_color = GLOBAL_RED_COLOR_CUT - 0.1
red_color = GLOBAL_RED_COLOR_CUT + 0.1

blue_dn = -1
red_dn = 3

results = is_quiescent_BGS_smart(np.array([7,8,9]), np.array([red_dn, np.nan, blue_dn]), np.array([blue_color, blue_color, red_color]))
assert results[0] == True
assert results[1] == False
assert results[2] == False

In [ ]:
# Basic test of multiple versions of SimpleRedshiftGuesser
# Ensure it handles arrays of inputs and gives a reasonable answer for a couple obvious cases

# Target (lost galaxies) properties
t_app_mag = np.array([19.0,18.0,12.0])
t_pobs = np.array([0.5, 0.5, 0.5])
t_q = np.array([True, True, False])

# Neighbor properties
nn_z = np.array([0.1, 0.2, 0.3])
nn_dist = np.array([250.0, 3.0, 30.0])
nn_q = np.array([True, True, False])

simple = SimpleRedshiftGuesser(None, None, ver='5.0')
z, nn_used = simple.choose_redshift(nn_z, nn_dist, t_pobs, t_app_mag, t_q, nn_q)

assert not nn_used[0]
assert nn_used[1]
assert not nn_used[2]
assert z[0] > 0.0
assert z[1] == 0.2
assert z[2] < 0.1

simple = SimpleRedshiftGuesser(None, None, ver='4.0')
z, nn_used = simple.choose_redshift(nn_z, nn_dist, t_pobs, t_app_mag, t_q, nn_q)

assert not nn_used[0]
assert nn_used[1]
assert not nn_used[2]
assert z[0] > 0.0
assert z[1] == 0.2
assert z[2] < 0.1

simple = SimpleRedshiftGuesser(None, None, ver='2.0')
z, nn_used = simple.choose_redshift(nn_z, nn_dist, t_pobs, t_app_mag, t_q, nn_q)

assert not nn_used[0]
assert nn_used[1]
assert not nn_used[2]
assert z[0] > 0.0
assert z[1] == 0.2
assert z[2] < 0.1


In [ ]:
# Basic tests for PhotometricRedshiftGuesser

# Target (lost galaxies) properties
t_app_mag = np.array([19.0,18.0,12.0])
t_pobs = np.array([0.5, 0.5, 0.5])
t_q = np.array([True, True, False])
t_zphot = np.array([0.105, 0.230, 0.011])

# Neighbor properties
nn_z = np.array([[0.1, 0.2, 0.3],
                 [0.2, 0.3, 0.1]])
nn_dist = np.array([[250.0, 3.0,  30.0],
                    [260.0, 40.0, 40.0]])
nn_q = np.array([[True, True, False],
                 [False, False, False]])

scorer = PhotometricRedshiftGuesser()
z, nn_used = scorer.choose_redshift(nn_z, nn_dist, t_zphot, t_pobs, t_app_mag, t_q, nn_q)

assert np.isnan(nn_used[0])
assert nn_used[1] == 0
assert np.isnan(nn_used[2])
assert z[0] > 0.0
assert z[1] == 0.2
assert z[2] < 0.1

In [ ]:
import numpy as np
x = [0, 1]
N = 3
lst = []
for i in range(N):
    lst.append(x)

coords_arrs = np.meshgrid(*lst)
np.shape(coords_arrs)
# iterate through each point
#for i in range(N):



In [ ]:
# Test my N-dimensional CIC binning function
data_2d = np.array([
    [0.0, 0.0],
    #[0.0, -0.5],
    [3.5, 3.5],
    #[2, 6.5],
    #[1.5, 2.1]
])
grid_size = (5, 5)

#binned_grid = cic_binning_2d(data_2d, grid_size)
#print(binned_grid)

first_dim  = np.linspace(0, 5, 6)
second_dim  = np.linspace(0, 6, 7)

bin_counts = cic_binning(data_2d, [first_dim, second_dim])
print(bin_counts)

assert np.shape(bin_counts) == (6, 7), np.shape(bin_counts)
assert np.sum(bin_counts) == len(data_2d), np.sum(bin_counts)
assert bin_counts[0,0] == 1.0, bin_counts[0,0]
assert bin_counts[3,3] == 0.25, bin_counts[3,3]
assert bin_counts[3,4] == 0.25, bin_counts[3,4]
assert bin_counts[4,3] == 0.25, bin_counts[4,3]
assert bin_counts[4,4] == 0.25, bin_counts[4,4]
